### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
from IPython.display import clear_output as clr
from sklearn.utils import class_weight
%matplotlib inline

### Setting working Directory

In [2]:
dir_path = (os.getcwd()+'\\').replace("\\","/")
print("Working Directory : ", dir_path)

Working Directory :  /media/abhikcr/New Volume/Study/M.tech 1st Sem/Cyber Security/Homeworks/KDD/extracted/


### Fucntions to Load files..

In [3]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [40]:
def readlines(fname, start, end):
#     fname = 'kddcup.data.corrected'
    file = open(fname)
    batch = []
    r = end - start
    for i in range(start):
        line = file.readline()
    for i in range(r):
        line = file.readline()
        batch.append(line.split(','))
    file.close()
    return batch

In [44]:
def rmreadlines(fname, size):
#     fname = 'kddcup.data.corrected'
    Nobs = file_len(fname)
    choices = np.sort(np.random.choice(Nobs,size, replace=False))
    
    k = 0
    
    file = open(fname)
    batch = []
    
    
    for i in range(Nobs):

        line = file.readline()
        if(k<choices.shape[0]):
            if(choices[k] == i):
                batch.append(line.split(','))
                k = k+1
            else:
                pass
        else:
            break;
    
    file.close()
    return batch

### Preprocessing Splitting data in train and test files

In [5]:
def split_file(fname, trainname, testname):
    Nobs = file_len(fname)
    tfp = int(Nobs*0.25)
    choices = np.sort(np.random.choice(Nobs,tfp, replace=False))
    k = 0
    file = open(fname)
    filetr = open(trainname, 'w')
    filets = open(testname, 'w')
    
    for i in range(Nobs):
        
        line = file.readline()
        if(k<choices.shape[0]):
            if(choices[k] == i):
                filets.write(line)
                k = k+1
            else:
                filetr.write(line)
        else:
            filetr.write(line)
            
    filets.close()
    filetr.close()
    file.close()
    return Nobs, choices.shape[0]

In [6]:
# Nobs, test_size = split_file('kddcup.data.corrected', 'train','test')

### Loading trainning data batchwise...

In [7]:
def myArrayConverter(arr):
    convertArr = []
    for s in arr.ravel():    
        try:
            value = float(s)
        except ValueError:
            value = s
        convertArr.append(value)
    return array(convertArr,dtype=object).reshape(arr.shape)

In [8]:
trainfile = 'train'
testfile = 'test'

In [9]:
train_size = file_len(trainfile)
print('Training data has',train_size, 'observations')

Training data has 3673824 observations


In [10]:
batch_size = 100000
n_batches = int(np.ceil(train_size/batch_size))

### Modeling...

In [11]:
label_list = pd.read_csv('training_attack_types', delimiter = ' ', index_col = False, names =['type', 'category'])

In [12]:
label_list = label_list.assign(id_type=(label_list['type']).astype('category').cat.codes)
label_list = label_list.assign(id_cat=(label_list['category']).astype('category').cat.codes)

In [13]:
label_list = label_list.append({'type':'normal', 'category':'normal','id_type':'-1','id_cat':'-1'}, ignore_index = True)

In [14]:
label_list['id_type'] = label_list['id_type'].astype(int)+1
label_list['id_cat'] = label_list['id_cat'].astype(int)+1

In [15]:
nClasses = label_list.shape[0]
label_list

,type,category,id_type,id_cat
0,back,dos,1,1
1,buffer_overflow,u2r,2,4
2,ftp_write,r2l,3,3
3,guess_passwd,r2l,4,3
4,imap,r2l,5,3
5,ipsweep,probe,6,2
6,land,dos,7,1
7,loadmodule,u2r,8,4
8,multihop,r2l,9,3
9,neptune,dos,10,1


In [16]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [17]:
import keras as kk

Using TensorFlow backend.


In [18]:
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko
import keras.regularizers as kr

In [45]:
currpos = 0
# batch = readlines(trainfile, currpos, currpos + batch_size)
batch = rmreadlines(trainfile, batch_size)


batchdf = pd.DataFrame(np.array(batch)).astype(float, errors='ignore')

for col in batchdf.columns:
    batchdf[col] = batchdf[col].astype('float64', errors = 'ignore') 

batcharr = batchdf.values

X = batcharr[:,:-1]
Y = batcharr[:,-1]

Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.\n'] = 0

Y_cat = Y_num.copy()
Y_type = Y_num.copy()
for i in label_list['type']:
    Y_cat[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_type']

Y_num_hot = one_hot(Y_num.astype(int), 2)
Y_cat_hot = one_hot(Y_cat.astype(int), 5)
Y_type_hot = one_hot(Y_type.astype(int), 23)

X_new_hot = np.delete(X,[1,2,3],1)

In [47]:
np.unique(Y)

array(['back.\n', 'ipsweep.\n', 'land.\n', 'loadmodule.\n', 'neptune.\n',
       'nmap.\n', 'normal.\n', 'pod.\n', 'portsweep.\n', 'satan.\n',
       'smurf.\n', 'spy.\n', 'teardrop.\n', 'warezclient.\n',
       'warezmaster.\n'], dtype=object)

In [20]:
cw_num = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_num),
                                                 Y_num)
cw_cat = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_cat),
                                                 Y_cat)
cw_type = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y_type),
                                                 Y_type)

cws_num = np.arange(2)*0.0
cws_num[np.unique(Y_num).astype(int)] = cw_num
# cws_num

cws_cat = np.arange(5)*0.0
cws_cat[np.unique(Y_cat).astype(int)] = cw_cat
# cws_cat

cws_type = np.arange(23)*0.0
cws_type[np.unique(Y_type).astype(int)] = cw_type
# cws_type

In [21]:
N,D = X_new_hot.shape

In [71]:
def get_model(input_shape):
    
    
    xin = kl.Input(shape = input_shape)
    x = kl.Dense(128, activation='relu',kernel_regularizer=kr.l2(0.01))(xin)
#     x = kl.Dense(64, activation='relu',kernel_regularizer=kr.l2(0.01))(x)
    x1 = kl.Dense(32, activation='relu',kernel_regularizer=kr.l2(0.01))(x)
    x1out = kl.Dense(2, activation='softmax', name = 'y1out',kernel_regularizer=kr.l2(0.01))(x1)
    
    x2 = kl.Dense(32, activation='relu',kernel_regularizer=kr.l2(0.01))(x)
    
    x2cat = kl.Concatenate(axis=-1)([x1out, x2])
    
    x2out = kl.Dense(5, activation='softmax', name = 'y2out',kernel_regularizer=kr.l2(0.01))(x2cat)
    
    x3cat = kl.Concatenate(axis=-1)([x2out, x])
    
    x3 = kl.Dense(32, activation='relu',kernel_regularizer=kr.l2(0.01))(x3cat)
    
    x3out = kl.Dense(23, activation='softmax', name = 'y3out',kernel_regularizer=kr.l2(0.01))(x3)
    
    model = km.Model(
        inputs=xin,
        outputs=[x1out,x2out,x3out],
        name="fashionnet")
    return model

In [72]:
model = get_model([D])

In [73]:
losses = {
    "y1out": "categorical_crossentropy",
    "y2out": "categorical_crossentropy",
    "y3out": "categorical_hinge",
}

In [74]:
lossWeights = {"y1out": 1.0, "y2out": 1.0, "y3out": 1.0}

In [75]:
opt = ko.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1.0e-9, amsgrad=False)

In [76]:
model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights, metrics=["accuracy"])

In [77]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 38)           0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 128)          4992        input_3[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 32)           4128        dense_10[0][0]                   
__________________________________________________________________________________________________
y1out (Dense)                   (None, 2)            66          dense_11[0][0]                   
__________________________________________________________________________________________________
dense_12 (

In [78]:
model.fit(X_new_hot,{"y1out": Y_num_hot, "y2out": Y_cat_hot, "y3out": Y_type_hot},epochs=1, batch_size = 1000
          ,class_weight={"y1out": cws_num, "y2out": cws_cat, "y3out": cws_type})

Epoch 1/1
100000/100000 [==============================] - 8s 85us/step - loss: 27.7379 - y1out_loss: 12.8486 - y2out_loss: 12.8186 - y3out_loss: 1.6063 - y1out_acc: 0.2026 - y2out_acc: 0.2041 - y3out_acc: 0.1967


In [ ]:
# model = km.load_model('model.h5')
for epochs in range(50):

    batch_size = 100000

    batch = rmreadlines(trainfile, batch_size)
    
    batchdf = pd.DataFrame(np.array(batch)).astype(float, errors='ignore')

    for col in batchdf.columns:
        batchdf[col] = batchdf[col].astype('float64', errors = 'ignore') 

    batcharr = batchdf.values

    X = batcharr[:,:-1]
    Y = batcharr[:,-1]

    
    print('\nBatch Size :', X.shape[0], 'at iteration :', epochs)
    
    Y_num = np.ones(Y.shape)
    Y_num[Y == 'normal.\n'] = 0

    Y_cat = Y_num.copy()
    Y_type = Y_num.copy()
    for i in label_list['type']:
        Y_cat[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_cat']
        Y_type[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_type']

    Y_num_hot = one_hot(Y_num.astype(int), 2)
    Y_cat_hot = one_hot(Y_cat.astype(int), 5)
    Y_type_hot = one_hot(Y_type.astype(int), 23)

    X_new_hot = np.delete(X,[1,2,3],1)

    print(np.unique(Y_num), np.unique(Y_cat),np.unique(Y_type))

    cw_num = class_weight.compute_class_weight('balanced',
                                         np.unique(Y_num),
                                         Y_num)
    cw_cat = class_weight.compute_class_weight('balanced',
                                                     np.unique(Y_cat),
                                                     Y_cat)
    cw_type = class_weight.compute_class_weight('balanced',
                                                     np.unique(Y_type),
                                                     Y_type)

    cws_num = np.arange(2)*0.0
    cws_num[np.unique(Y_num).astype(int)] = cw_num
    # cws_num

    cws_cat = np.arange(5)*0.0
    cws_cat[np.unique(Y_cat).astype(int)] = cw_cat
    # cws_cat

    cws_type = np.arange(23)*0.0
    cws_type[np.unique(Y_type).astype(int)] = cw_type

    if(X_new_hot.shape[1]!=D or np.unique(Y).shape[0] == 1):
        continue;


    model.fit(X_new_hot,{"y1out": Y_num_hot, "y2out": Y_cat_hot, "y3out": Y_type_hot},epochs=1, batch_size=1000
             ,class_weight={"y1out": cws_num, "y2out": cws_cat, "y3out": cws_type}) 
    if(epochs%10 == 0):
        clr()
        model.save('model.h5')


Batch Size : 100000 at iteration : 1
[0. 1.] [0. 1. 2. 3. 4.] [ 0.  1.  2.  6.  8. 10. 11. 14. 15. 17. 18. 20. 21. 22.]
Epoch 1/1
100000/100000 [==============================] - 6s 65us/step - loss: 0.2483 - y1out_loss: 0.0324 - y2out_loss: 0.0555 - y3out_loss: 0.0235 - y1out_acc: 0.9966 - y2out_acc: 0.9914 - y3out_acc: 0.9885

Batch Size : 100000 at iteration : 2
[0. 1.] [0. 1. 2. 3. 4.] [ 0.  1.  2.  4.  6. 10. 11. 14. 15. 16. 17. 18. 20. 21.]
Epoch 1/1
100000/100000 [==============================] - 8s 77us/step - loss: 1.7745 - y1out_loss: 0.5388 - y2out_loss: 0.2556 - y3out_loss: 0.6522 - y1out_acc: 0.9611 - y2out_acc: 0.9729 - y3out_acc: 0.6736

Batch Size : 100000 at iteration : 3
[0. 1.] [0. 1. 2. 3.] [ 0.  1.  3.  6.  9. 10. 11. 14. 15. 17. 18. 20. 21.]
Epoch 1/1
100000/100000 [==============================] - 6s 63us/step - loss: 2.2899 - y1out_loss: 0.1423 - y2out_loss: 1.1473 - y3out_loss: 0.3593 - y1out_acc: 0.9904 - y2out_acc: 0.9259 - y3out_acc: 0.8204

Batch Size : 